In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

n_mean = 0.5
n_std = 0.5
n_epochs = 2
batch_size = 4
num_workers = 2

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((n_mean, n_mean, n_mean), (n_std, n_std, n_std))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net = nn.Sequential(
   nn.Conv2d(3,6,5),
   nn.ReLU(),
   nn.Conv2d(6,10,5),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(5760,10),
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001)

for epoch in range(n_epochs):
    for data in trainloader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# uncomment to save
# PATH = './cifar_net.pth'
# torch.save(net.state_dict(), PATH)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: ', correct/total)

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(batch_size):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Files already downloaded and verified
Files already downloaded and verified
Accuracy:  0.4814
Accuracy of plane : 33 %
Accuracy of   car : 57 %
Accuracy of  bird : 28 %
Accuracy of   cat : 42 %
Accuracy of  deer : 48 %
Accuracy of   dog : 16 %
Accuracy of  frog : 60 %
Accuracy of horse : 53 %
Accuracy of  ship : 81 %
Accuracy of truck : 58 %
